In [ ]:
import mne
import numpy as np
import utils

In [ ]:
"""LEGACY SCRIPT FOR WORKING WITH NEONATAL DATA"""

In [ ]:
##load data for patient
patient_number = 4
file = rf"neonatal_data\eeg{patient_number}.edf"
data_full = mne.io.read_raw_edf(file,preload = True,verbose=False)
channels_original = data_full.ch_names
## copy the data to apply filtering without including the ekg and respiratory channels
data_eeg = data_full.copy()
montage = mne.channels.read_custom_montage(r"A:\Program Files\MATLAB\R2020a\eeglab2022.1\my_locs\neonatal_channel_loc_mne.loc")
new_channels = {}
montage_channels = montage.ch_names
for n in range (0,len(channels_original)-2):
    new_channels[channels_original[n]] = montage_channels[n]
## add electrode positions
data_eeg.rename_channels(new_channels)
data_eeg.set_montage(montage,on_missing = 'ignore')
## remove respiraotry and ekg channels
if patient_number == 1:
    data_eeg.drop_channels(['ECG EKG-REF','Resp Effort-REF'])
else:
    data_eeg.drop_channels(['ECG EKG','Resp Effort'])